In [86]:
import numpy as np
import pandas as pd

In [30]:
### Data Load
# test_ratings.csv == sample_submission.csv

In [4]:
DATA_PATH = 'data/'

users = pd.read_csv(DATA_PATH + 'users.csv')
books = pd.read_csv(DATA_PATH + 'books.csv')
train = pd.read_csv(DATA_PATH + 'train_ratings.csv')
test = pd.read_csv(DATA_PATH + 'test_ratings.csv')
sub = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [8]:
users.head(3)

,user_id,location,age
0,8,"timmins, ontario, canada",NaN
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",NaN


In [48]:
books.head(2)

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,1,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg


In [10]:
train.head(3)

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8


In [11]:
test.head(3)

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0


In [12]:
sub.head(3)

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0


### context_data.py

##### context_data_load()

In [50]:
# idx 만들 때, train과 sub의 데이터 concat해서 진행
ids = pd.concat([train['user_id'], sub['user_id']]).unique()
isbns = pd.concat([train['isbn'], sub['isbn']]).unique()
ids, isbns

(array([    0,     1,     2, ..., 68066, 68067, 68068]),
 array([     0,      1,      2, ..., 149567, 149568, 149569]))

In [23]:
idx2user = {idx:id for idx, id in enumerate(ids)}
user2idx = {id:idx for idx, id in idx2user.items()}
for i in range(3):
    print(idx2user[i], end='\t')
    print(user2idx[idx2user[i]], end='\t')

8	0	67544	1	123629	2	

In [26]:
idx2isbn = {idx:isbn for idx, isbn in enumerate(isbns)}
isbn2idx = {isbn:idx for idx, isbn in idx2isbn.items()}

print(idx2isbn[0], isbn2idx[idx2isbn[0]])

0002005018 0


In [27]:
train['user_id']

0              8
1          67544
2         123629
3         200273
4         210926
           ...  
306790    278843
306791    278851
306792    278851
306793    278851
306794    278851
Name: user_id, Length: 306795, dtype: int64

In [28]:
train['user_id'] = train['user_id'].map(user2idx)
train['user_id']

0            0
1            1
2            2
3            3
4            4
          ... 
306790    6313
306791    1879
306792    1879
306793    1879
306794    1879
Name: user_id, Length: 306795, dtype: int64

In [29]:
sub['user_id'] = sub['user_id'].map(user2idx)
test['user_id'] = test['user_id'].map(user2idx)
users['user_id'] = users['user_id'].map(user2idx)

In [31]:
train['isbn']

0         0002005018
1         0002005018
2         0002005018
3         0002005018
4         0002005018
             ...    
306790    0743525493
306791    067161746X
306792    0884159221
306793    0912333022
306794    1569661057
Name: isbn, Length: 306795, dtype: object

In [32]:
train['isbn'] = train['isbn'].map(isbn2idx)
train['isbn']

0              0
1              0
2              0
3              0
4              0
           ...  
306790    129772
306791    129773
306792    129774
306793    129775
306794    129776
Name: isbn, Length: 306795, dtype: int64

In [37]:
sub['isbn'] = sub['isbn'].map(isbn2idx)
test['isbn'] = test['isbn'].map(isbn2idx)
books['isbn'] = books['isbn'].map(isbn2idx)

##### process_context_data(users, books, ratings1=train, ratings2=test)

In [38]:
# location 쪼개기
users['location_city'] = users['location'].apply(lambda x: x.split(',')[0])
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1])
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2])
users = users.drop(['location'], axis=1)

In [41]:
users.head(3)

,user_id,age,location_city,location_state,location_country
0,0.0,NaN,timmins,ontario,canada
1,2334.0,49.0,ottawa,ontario,canada
2,13.0,NaN,n/a,n/a,n/a


In [44]:
# train, test 합쳐버림
ratings = pd.concat([train, test]).reset_index(drop=True)
ratings

,user_id,isbn,rating
0,0,0,4
1,1,0,7
2,2,0,8
3,3,0,8
4,4,0,9
...,...,...,...
383489,7728,149565,0
383490,47785,149566,0
383491,4209,149567,0
383492,40779,149568,0


In [51]:
# book_title / year_of_publication / summary / img_url / img_path 제외
context_df = ratings.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')

In [70]:
context_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383494 entries, 0 to 383493
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           383494 non-null  int64  
 1   isbn              383494 non-null  int64  
 2   rating            383494 non-null  int64  
 3   age               267707 non-null  float64
 4   location_city     383494 non-null  object 
 5   location_state    383494 non-null  object 
 6   location_country  383494 non-null  object 
 7   category          231954 non-null  object 
 8   publisher         383494 non-null  object 
 9   language          234620 non-null  object 
 10  book_author       383494 non-null  object 
dtypes: float64(1), int64(3), object(7)
memory usage: 35.1+ MB


In [59]:
train_df = train.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')
test_df = test.merge(users, on='user_id', how='left').merge(books[['isbn', 'category', 'publisher', 'language', 'book_author']], on='isbn', how='left')

In [60]:
loc_city2idx = {v:k for k,v in enumerate(context_df['location_city'].unique())}
loc_state2idx = {v:k for k,v in enumerate(context_df['location_state'].unique())}
loc_country2idx = {v:k for k,v in enumerate(context_df['location_country'].unique())}

In [71]:
# indexing location feautures
train_df['location_city'] = train_df['location_city'].map(loc_city2idx)
train_df['location_state'] = train_df['location_state'].map(loc_state2idx)
train_df['location_country'] = train_df['location_country'].map(loc_country2idx)
test_df['location_city'] = test_df['location_city'].map(loc_city2idx)
test_df['location_state'] = test_df['location_state'].map(loc_state2idx)
test_df['location_country'] = test_df['location_country'].map(loc_country2idx)

##### age_map(x: int)

In [72]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

In [73]:
# age 결측값 mean으로 처리하고, age feature 이산화
train_df['age'] = train_df['age'].fillna(int(train_df['age'].mean()))
train_df['age'] = train_df['age'].apply(age_map)
test_df['age'] = test_df['age'].fillna(int(test_df['age'].mean()))
test_df['age'] = test_df['age'].apply(age_map)

In [74]:
train_df['age']

0         3
1         3
2         3
3         3
4         3
         ..
306790    2
306791    3
306792    3
306793    3
306794    3
Name: age, Length: 306795, dtype: int64

In [75]:
# nan는 index 값 하나로 퉁침
context_df['language'].unique()

array(['en', nan, 'de', 'fr', 'es', 'ca', 'it', 'da', 'nl', 'pt', 'ro',
       'no', 'ms', 'la', 'ru', 'el', 'zh-CN', 'ja', 'fa', 'eo', 'ga',
       'vi', 'zh-TW', 'ko', 'gl', 'th', 'gd'], dtype=object)

In [76]:
category2idx = {v:k for k,v in enumerate(context_df['category'].unique())}
publisher2idx = {v:k for k,v in enumerate(context_df['publisher'].unique())}
language2idx = {v:k for k,v in enumerate(context_df['language'].unique())}
author2idx = {v:k for k,v in enumerate(context_df['book_author'].unique())}

In [80]:
train_df['category'] = train_df['category'].map(category2idx)
train_df['publisher'] = train_df['publisher'].map(publisher2idx)
train_df['language'] = train_df['language'].map(language2idx)
train_df['book_author'] = train_df['book_author'].map(author2idx)
test_df['category'] = test_df['category'].map(category2idx)
test_df['publisher'] = test_df['publisher'].map(publisher2idx)
test_df['language'] = test_df['language'].map(language2idx)
test_df['book_author'] = test_df['book_author'].map(author2idx)

In [82]:
idx = {
    "loc_city2idx":loc_city2idx,
    "loc_state2idx":loc_state2idx,
    "loc_country2idx":loc_country2idx,
    "category2idx":category2idx,
    "publisher2idx":publisher2idx,
    "language2idx":language2idx,
    "author2idx":author2idx,
}
context_train = train_df
context_test = test_df

##### End of process_context_data()

In [84]:
context_train['age'].unique()

array([3, 6, 2, 5, 4, 1])

In [87]:
field_dims = np.array([len(user2idx), len(isbn2idx),
                            6, len(idx['loc_city2idx']), len(idx['loc_state2idx']), len(idx['loc_country2idx']),
                            len(idx['category2idx']), len(idx['publisher2idx']), len(idx['language2idx']), len(idx['author2idx'])], dtype=np.uint32)

In [88]:
data = {
    'train':context_train,
    'test':context_test.drop(['rating'], axis=1),
    'field_dims':field_dims,
    'users':users,
    'books':books,
    'sub':sub,
    'idx2user':idx2user,
    'idx2isbn':idx2isbn,
    'user2idx':user2idx,
    'isbn2idx':isbn2idx,
}

##### End of context_data_load()